# 🏛️ Chapter 9: Advanced Game Architecture (Building an Engine)

Welcome to the Big Leagues. 🧢

Small beginner projects can live in a single `main.py` file. But if you want to build an RPG, a platformer with 50 levels, or a complex strategy game, that approach will collapse.

In this chapter, we stop writing scripts and start building an **Engine**. We will implement:
1.  **The Game Class**: The brain that holds everything.
2.  **The State Stack**: Managing transitions (Menu -> Game -> Pause).
3.  **Delta Time**: Ensuring the game runs at the same speed on *any* computer.

## 1. The `Game` Singleton

Instead of global variables (`screen`, `clock`, `running`), we wrap everything in a robust class.

In [ ]:
import pygame
import sys

class Game:
    def __init__(self):
        pygame.init()
        self.screen = pygame.display.set_mode((800, 600))
        self.clock = pygame.time.Clock()
        self.running = True
        self.states = [] # The State Stack!
        
    def run(self):
        while self.running:
            # 1. Get Delta Time (dt)
            # dt is seconds since last frame (e.g., 0.016 for 60fps)
            dt = self.clock.tick(60) / 1000.0
            
            # 2. Event Handling
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                
                # Pass events to the TOP state
                if self.states:
                    self.states[-1].process_event(event)
            
            # 3. Update & Draw
            if self.states:
                self.states[-1].update(dt)
                self.states[-1].draw(self.screen)
                
            pygame.display.flip()
        
        pygame.quit()
        sys.exit()

## 2. The Abstract State Class

Every "Scene" in your game (MainMenu, Level1, Inventory, Credits) follows the same blueprint.

In [ ]:
class State:
    def __init__(self, game):
        self.game = game
        
    def process_event(self, event):
        pass
        
    def update(self, dt):
        pass
        
    def draw(self, screen):
        pass
    
    def enter(self):
        print(f"Entered {self.__class__.__name__}")
        
    def exit(self):
        print(f"Exited {self.__class__.__name__}")

## 3. Creating a Real Menu State

Now, let's make a Menu that launches the Game.

In [ ]:
class MenuState(State):
    def draw(self, screen):
        screen.fill((50, 0, 50)) # Dark Purple
        # In real code, you'd use your FontManager here!
        
    def process_event(self, event):
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN:
                # Pushing the PlayState on top!
                new_state = PlayState(self.game)
                self.game.states.append(new_state)

class PlayState(State):
    def draw(self, screen):
        screen.fill((0, 100, 0)) # Green Field
        
    def process_event(self, event):
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                # Pop self off the stack, returning to Menu
                self.game.states.pop()

## 🛠️ The Challenge: Implementing the Engine

1.  Copy the `Game` and `State` classes.
2.  Create a `TitleScreen` state.
3.  Create a `Gameplay` state.
4.  Initialize the `Game` object, push the `TitleScreen`, and call `run()`.

If you master this, you have the foundation for *any* 2D game you can imagine. 🚀